In [1]:
%load_ext autoreload

In [2]:
import sys
sys.path.append("..") #this is to add the above folder to the package directory
import os
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas import ExcelWriter

# from pandas.plotting import register_matplotlib_converters
import plotly.express as px
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import cufflinks as cf
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot

In [3]:
%matplotlib inline 

In [4]:
init_notebook_mode(connected=True) #this will connect plotly to the java script

In [5]:
cf.go_offline() #this will allow you to use cufflinks offline

### Reading the file and seting the dataframe


In [6]:
folder_path = os.path.join('..','Data', 'hydrological_data')
hourly_data = os.path.join(folder_path, 'New rules Fierza 20210401.xlsx') #This dataframe represnts actual data measured by operatores
df = pd.read_excel(hourly_data,sheet_name='opr_measure')
opr_rules = os.path.join(folder_path, 'New rules Fierza 20210401.xlsx') #This dataframe represnts modeling data. Becarful when you compare it with the other dataframe.
df_opr = pd.read_excel(opr_rules,sheet_name='weekly')
curve_path = os.path.join(folder_path, 'level_volume_curves.xlsx')
fier_curve = pd.read_excel(curve_path, sheet_name='fierza')

In [7]:
#df_opr = df_opr[['month', 'week', 'Fierza lake outflow', 'Max Level (m)',
#       'Current Operational level (m)', 'New operational level (m)']]
#df_opr.set_index('week',inplace=True)
df_opr.head()

,month,week,fierza_lake_outflow,max_level(m),hist_level(m),max_storage(MCM),hist_storage(MCM),opt_level(m),opt_storage(MCM),hist_buffer(MCM),opt_buffer(MCM)
0,1,1,320.475183,290,279,2693,1658,275,1470,1035,1223
1,1,2,316.460066,290,279,2693,1658,275,1470,1035,1223
2,1,3,274.732042,290,279,2693,1658,275,1470,1035,1223
3,1,4,209.738762,290,279,2693,1658,275,1470,1035,1223
4,1,5,243.402371,290,279,2693,1658,275,1470,1035,1223


In [8]:
df2 = df.copy()
#df2['Date']=df2['Date']
df2['year'] = df2['Date'].dt.year 
df2['month'] = df2['Date'].dt.month
df2['week'] = df2['Date'].dt.week
df2.head()

<ipython-input-8-883ceb804bf7>:5: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



,Date,water level [m],inflow [m3/s],inflow [m3/s].1,spillway outflow [milion m3],year,month,week
0,2003-01-01,283.84,168,179.0,0.0,2003,1,1
1,2003-01-02,283.72,160,168.0,0.0,2003,1,1
2,2003-01-03,284.30,223,224.0,0.0,2003,1,1
3,2003-01-04,285.00,535,549.0,0.0,2003,1,1
4,2003-01-05,286.15,600,683.0,0.0,2003,1,1


In [9]:
df2.columns

Index(['Date', 'water level [m]', 'inflow [m3/s]', 'inflow [m3/s].1',
       'spillway outflow [milion m3]', 'year', 'month', 'week'],
      dtype='object')

In [10]:
df3=df2.groupby(['week'])['water level [m]', 'inflow [m3/s]', 'inflow [m3/s].1',
       'spillway outflow [milion m3]'].mean().reset_index()


<ipython-input-10-4ba36b67ad00>:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [11]:
df3 = df3[df3.week != 53]

In [12]:
dff = pd.merge(df3, df_opr, on='week')
dff.reset_index(inplace=True)

In [13]:
dff['LowerLevel (m)'] = 176
#dff.reindex(columns=['New Operational level (m)'])
dff.head()

,index,week,water level [m],inflow [m3/s],inflow [m3/s].1,spillway outflow [milion m3],month,fierza_lake_outflow,max_level(m),hist_level(m),max_storage(MCM),hist_storage(MCM),opt_level(m),opt_storage(MCM),hist_buffer(MCM),opt_buffer(MCM),LowerLevel (m)
0,0,1,279.898052,252.714286,254.983566,4.020779,1,320.475183,290,279,2693,1658,275,1470,1035,1223,176
1,1,2,279.647662,245.233766,248.034306,1.432987,1,316.460066,290,279,2693,1658,275,1470,1035,1223,176
2,2,3,278.751818,204.246753,204.097167,0.000000,1,274.732042,290,279,2693,1658,275,1470,1035,1223,176
3,3,4,278.837532,272.935065,273.442091,0.000000,1,209.738762,290,279,2693,1658,275,1470,1035,1223,176
4,4,5,279.090130,255.675325,254.971526,0.000000,1,243.402371,290,279,2693,1658,275,1470,1035,1223,176


In [14]:
fier_df = df_opr

fier_df.head()

,month,week,fierza_lake_outflow,max_level(m),hist_level(m),max_storage(MCM),hist_storage(MCM),opt_level(m),opt_storage(MCM),hist_buffer(MCM),opt_buffer(MCM)
0,1,1,320.475183,290,279,2693,1658,275,1470,1035,1223
1,1,2,316.460066,290,279,2693,1658,275,1470,1035,1223
2,1,3,274.732042,290,279,2693,1658,275,1470,1035,1223
3,1,4,209.738762,290,279,2693,1658,275,1470,1035,1223
4,1,5,243.402371,290,279,2693,1658,275,1470,1035,1223


In [15]:
fier_df.columns

Index(['month', 'week', 'fierza_lake_outflow', 'max_level(m)', 'hist_level(m)',
       'max_storage(MCM)', 'hist_storage(MCM)', 'opt_level(m)',
       'opt_storage(MCM)', 'hist_buffer(MCM)', 'opt_buffer(MCM)'],
      dtype='object')

In [16]:
# Flood Protection Scenario

levels = [5, 10, 15, 20]
Fierza_power = 500
density = 1000 #(kg/m3) = 
g = 9.81 #(m/sec2) 
Turb_eff = 0.9 #
fierza_lower_level = 176

for level in levels:  
    # Create the new dataframe
    _df = df_opr[['month', 'week', 'max_level(m)','hist_level(m)', 'max_storage(MCM)', 'hist_storage(MCM)','hist_buffer(MCM)']].copy()
    
    # Calculating new buffer and storage
    _df['new_buffer(MCM)'] = _df['hist_buffer(MCM)'] + (_df['hist_buffer(MCM)']*(level/100))
    _df.loc[(_df['month']>4) & (_df['month']<10), 'new_buffer(MCM)'] = _df['hist_buffer(MCM)']
    
    _df['new_storage(MCM)'] = _df['max_storage(MCM)'] - _df['new_buffer(MCM)']
    
    _df['changeinbuffer(MCM)'] = (_df['new_buffer(MCM)'] - _df['hist_buffer(MCM)'])/_df['hist_buffer(MCM)']
    _df['additional_buffer(MCM)'] = (_df['new_buffer(MCM)'] - _df['hist_buffer(MCM)'])
    
    # Reading the new water level based on the volume-elevation curve
    level_predict = np.poly1d(np.polyfit(fier_curve['Volume(MCM)'], fier_curve['Level(m)'], 2))
    _df['new_level(m)'] = level_predict(_df['new_storage(MCM)'].round(0))
    _df.loc[(_df['month']>4) & (_df['month']<10), 'new_level(m)'] = _df['hist_level(m)']
    
    _df['lowerlevel(m)'] = fierza_lower_level
    # Add the calculations of X for each week
    _df['hist_head(m)'] = _df['hist_level(m)'] - _df['lowerlevel(m)']
    _df['hist_Q(m/sec)'] = ((Fierza_power*1000000)/(density*g*Turb_eff*_df['hist_head(m)']))
    _df['hist_flow(MCM)'] = (_df['hist_Q(m/sec)']*3600*8760)/1000000
    _df['hist_energy(PJ)'] = ((Fierza_power/1000)*31.536)
    _df['hist_X(MCM/PJ)'] = _df['hist_flow(MCM)']/_df['hist_energy(PJ)']
    
    _df['new_head(m)'] = _df['new_level(m)'] - _df['lowerlevel(m)']
    _df['new_Q(m/sec)'] = ((Fierza_power*1000000)/(density*g*Turb_eff*_df['new_head(m)']))
    _df['new_flow(MCM)'] = (_df['new_Q(m/sec)']*3600*8760)/1000000
    _df['new_energy(PJ)'] = ((Fierza_power/1000)*31.536)
    _df['new_X(MCM/PJ)'] = _df['new_flow(MCM)']/_df['new_energy(PJ)']
    _df['mode'] = 0
    # Generate the outputfile that goes to osemosys
    hist_cap = _df[['month','week','hist_storage(MCM)']]
    new_cap = _df[['month','week','new_storage(MCM)']]
    hist_x = _df[['week','mode','hist_X(MCM/PJ)']]
    new_x = _df[['week','mode','new_X(MCM/PJ)']]
    rule = _df.groupby('month')[['hist_level(m)','new_level(m)','additional_buffer(MCM)']].mean().round(1).reset_index()
    rule['diff'] = rule['hist_level(m)'] - rule['new_level(m)']
    
    #res_cap = ref[['week','Utilized Volume (MCM)']]
    for i in range(20,56):
        hist_cap['20{}'.format(i)]=hist_cap['hist_storage(MCM)']
        new_cap['20{}'.format(i)]=new_cap['new_storage(MCM)']
    hist_cap.drop('hist_storage(MCM)', axis=1, inplace=True)
    hist_cap.rename({'2055':'2055:=','week':' '},axis=1, inplace=True)
    new_cap.drop('new_storage(MCM)', axis=1, inplace=True)
    new_cap.rename({'2055':'2055:=','week':' '},axis=1, inplace=True)
    
    
    # Save files
    output_folder = os.path.join('..','processed_data', 'flood protection scenario', f'{level}%_change_buffer')
    os.makedirs(output_folder, exist_ok = True)

    file_path=os.path.join(output_folder, f'Fierza_at_{level}%buffer.xlsx')
    writer = pd.ExcelWriter(file_path, engine='xlsxwriter')
    
    
    _df.round(2).to_excel(writer, index=False, sheet_name='all')
    hist_cap.round(2).to_excel(writer, index=False, sheet_name='hist_cap')
    new_cap.round(2).to_excel(writer, index=False, sheet_name='new_cap')
    hist_x.round(2).to_excel(writer, index=False, sheet_name='hist_x')
    new_x.round(2).to_excel(writer, index=False, sheet_name='new_x')
    rule.round(2).to_excel(writer, index=False, sheet_name='rule')

    writer.save()
    


In [54]:
# Energy Optimization scenario
Fierza_power = 500
density = 1000 #(kg/m3) = 
g = 9.81 #(m/sec2) 
Turb_eff = 0.9 #
fierza_lower_level = 176

# Create the new dataframe
_dff = df_opr[['month', 'week', 'max_level(m)','hist_level(m)', 'max_storage(MCM)', 'hist_storage(MCM)','hist_buffer(MCM)']].copy()

# Calculating new buffer and storage
_dff['eng_level(m)'] = _dff['max_level(m)']

# Reading the new water level based on the volume-elevation curve
level_predict = np.poly1d(np.polyfit(fier_curve['Level(m)'], fier_curve['Volume(MCM)'], 5))
_dff['eng_storage(MCM)'] = level_predict(_dff['eng_level(m)'].round(1))

_dff['eng_buffer(MCM)'] = _dff['max_storage(MCM)'] - _dff['eng_storage(MCM)']

_dff['changeinbuffer(%)'] = 100*(_dff['eng_buffer(MCM)'] - _dff['hist_buffer(MCM)'])/_dff['hist_buffer(MCM)']
_dff['changeinstorage(%)'] = 100*(_dff['eng_storage(MCM)'] - _dff['hist_storage(MCM)'])/_dff['hist_storage(MCM)']

_dff['lowerlevel(m)'] = fierza_lower_level
# Add the calculations of X for each week
_dff['hist_head(m)'] = _dff['hist_level(m)'] - _dff['lowerlevel(m)']
_dff['hist_Q(m/sec)'] = ((Fierza_power*1000000)/(density*g*Turb_eff*_dff['hist_head(m)']))
_dff['hist_flow(MCM)'] = (_dff['hist_Q(m/sec)']*3600*8760)/1000000
_dff['hist_energy(PJ)'] = ((Fierza_power/1000)*31.536)
_dff['hist_X(MCM/PJ)'] = _dff['hist_flow(MCM)']/_dff['hist_energy(PJ)']

_dff['eng_head(m)'] = _dff['eng_level(m)'] - _dff['lowerlevel(m)']
_dff['eng_Q(m/sec)'] = ((Fierza_power*1000000)/(density*g*Turb_eff*_dff['eng_head(m)']))
_dff['eng_flow(MCM)'] = (_dff['eng_Q(m/sec)']*3600*8760)/1000000
_dff['eng_energy(PJ)'] = ((Fierza_power/1000)*31.536)
_dff['eng_X(MCM/PJ)'] = _dff['eng_flow(MCM)']/_dff['eng_energy(PJ)']
_dff['mode'] = 0

# Generate the outputfile that goes to osemosys
hist_cap = _dff[['month','week','hist_storage(MCM)']]
eng_cap = _dff[['month','week','eng_storage(MCM)']]
hist_x = _dff[['week','mode','hist_X(MCM/PJ)']]
eng_x = _dff[['week','mode','eng_X(MCM/PJ)']]
rule = _dff.groupby('month')[['hist_level(m)','eng_level(m)']].mean().round(1).reset_index()
rule['diff(m)'] = rule['eng_level(m)'] - rule['hist_level(m)']
rule['hpp'] = 'fierza'

#res_cap = ref[['week','Utilized Volume (MCM)']]
for i in range(20,56):
    hist_cap['20{}'.format(i)]=hist_cap['hist_storage(MCM)']
    eng_cap['20{}'.format(i)]=eng_cap['eng_storage(MCM)']
hist_cap.drop('hist_storage(MCM)', axis=1, inplace=True)
hist_cap.rename({'2055':'2055:=','week':' '},axis=1, inplace=True)
eng_cap.drop('eng_storage(MCM)', axis=1, inplace=True)
eng_cap.rename({'2055':'2055:=','week':' '},axis=1, inplace=True)


# Save files
output_folder = os.path.join('..','processed_data', 'energy optimisation scenario')
os.makedirs(output_folder, exist_ok = True)

file_path=os.path.join(output_folder, 'Fierza_energy_optimisation.xlsx')
writer = pd.ExcelWriter(file_path, engine='xlsxwriter')


_dff.round(2).to_excel(writer, index=False, sheet_name='all')
hist_cap.round(2).to_excel(writer, index=False, sheet_name='hist_cap')
eng_cap.round(2).to_excel(writer, index=False, sheet_name='eng_cap')
hist_x.round(2).to_excel(writer, index=False, sheet_name='hist_x')
eng_x.round(2).to_excel(writer, index=False, sheet_name='eng_x')
rule.round(2).to_excel(writer, index=False, sheet_name='rule')

writer.save()


In [55]:
_dff.round(0)

,month,week,max_level(m),hist_level(m),max_storage(MCM),hist_storage(MCM),hist_buffer(MCM),eng_level(m),eng_storage(MCM),eng_buffer(MCM),...,hist_Q(m/sec),hist_flow(MCM),hist_energy(PJ),hist_X(MCM/PJ),eng_head(m),eng_Q(m/sec),eng_flow(MCM),eng_energy(PJ),eng_X(MCM/PJ),mode
0,1,1,290,279,2693,1658,1035,290,2278.0,415.0,...,550.0,17339.0,16.0,1100.0,114,497.0,15666.0,16.0,994.0,0
1,1,2,290,279,2693,1658,1035,290,2278.0,415.0,...,550.0,17339.0,16.0,1100.0,114,497.0,15666.0,16.0,994.0,0
2,1,3,290,279,2693,1658,1035,290,2278.0,415.0,...,550.0,17339.0,16.0,1100.0,114,497.0,15666.0,16.0,994.0,0
3,1,4,290,279,2693,1658,1035,290,2278.0,415.0,...,550.0,17339.0,16.0,1100.0,114,497.0,15666.0,16.0,994.0,0
4,1,5,290,279,2693,1658,1035,290,2278.0,415.0,...,550.0,17339.0,16.0,1100.0,114,497.0,15666.0,16.0,994.0,0
5,2,6,290,276,2693,1515,1178,290,2278.0,415.0,...,566.0,17859.0,16.0,1133.0,114,497.0,15666.0,16.0,994.0,0
6,2,7,290,276,2693,1515,1178,290,2278.0,415.0,...,566.0,17859.0,16.0,1133.0,114,497.0,15666.0,16.0,994.0,0
7,2,8,290,276,2693,1515,1178,290,2278.0,415.0,...,566.0,17859.0,16.0,1133.0,114,497.0,15666.0,16.0,994.0,0
8,2,9,290,276,2693,1515,1178,290,2278.0,415.0,...,566.0,17859.0,16.0,1133.0,114,497.0,15666.0,16.0,994.0,0
9,3,10,292,280,2693,1707,986,292,2410.0,283.0,...,545.0,17172.0,16.0,1089.0,116,488.0,15396.0,16.0,976.0,0


In [16]:
rule['hpp'] = 'fierza'
rule

,month,hist_level(m),new_level(m),diff,hpp
0,1,279,274.8,4.2,fierza
1,2,276,270.2,5.8,fierza
2,3,280,276.2,3.8,fierza
3,4,285,283.3,1.7,fierza
4,5,290,290.0,0.0,fierza
5,6,296,296.0,0.0,fierza
6,7,293,293.0,0.0,fierza
7,8,286,286.0,0.0,fierza
8,9,275,275.0,0.0,fierza
9,10,272,264.2,7.8,fierza


In [ ]:
# old code continues here

In [26]:
ref = dff[['week','Max Level (m)','Current Operational level (m)','LowerLevel (m)']].copy()
new = dff[['week','Max Level (m)','New operational level (m)','LowerLevel (m)']].copy()

In [27]:
Skavic_power = 196
Fierza_power = 500
Komani_power = 600
Vaudej_power = 250
density = 1000 #(kg/m3) = 
g = 9.81 #(m/sec2)  
Turb_eff = 0.9 # 


In [28]:
ref['Head (m)'] = ref['Current Operational level (m)'] - ref['LowerLevel (m)']
ref['Q [m/sec]'] = ((Fierza_power*1000000)/(density*g*Turb_eff*ref['Head (m)']))
ref['flow_MCM'] = (ref['Q [m/sec]']*3600*8760)/1000000
ref['energy_PJ'] = ((Fierza_power/1000)*31.536)
ref['ratio_X [MCM/PJ]'] = ref['flow_MCM']/ref['energy_PJ']
#dff['res_cap [MCM]'] = ((dff['inflow [m3/s].1'].mean()*3600*8760)/1000000)

In [29]:
new['Head (m)'] = new['New operational level (m)'] - new['LowerLevel (m)']
new['Q [m/sec]'] = ((Fierza_power*1000000)/(density*g*Turb_eff*new['Head (m)']))
new['flow_MCM'] = (new['Q [m/sec]']*3600*8760)/1000000
new['energy_PJ'] = ((Fierza_power/1000)*31.536)
new['ratio_X [MCM/PJ]'] = new['flow_MCM']/new['energy_PJ']


In [30]:
new

,week,Max Level (m),New operational level (m),LowerLevel (m),Head (m),Q [m/sec],flow_MCM,energy_PJ,ratio_X [MCM/PJ]
0,1,290,275,176,99,572.035910,18039.724462,15.768,1144.071820
1,2,290,275,176,99,572.035910,18039.724462,15.768,1144.071820
2,3,290,275,176,99,572.035910,18039.724462,15.768,1144.071820
3,4,290,275,176,99,572.035910,18039.724462,15.768,1144.071820
4,5,290,275,176,99,572.035910,18039.724462,15.768,1144.071820
5,6,290,271,176,95,596.121633,18799.291808,15.768,1192.243265
6,7,290,271,176,95,596.121633,18799.291808,15.768,1192.243265
7,8,290,271,176,95,596.121633,18799.291808,15.768,1192.243265
8,9,290,271,176,95,596.121633,18799.291808,15.768,1192.243265
9,10,292,272,176,96,589.912032,18603.465851,15.768,1179.824065


In [31]:
output_folder = os.path.join('..','processed_data', 'flood protection scenario')
os.makedirs(output_folder, exist_ok = True)

file_path=os.path.join(output_folder, 'Reference_scenario_20210401.xlsx')
writer1 = pd.ExcelWriter(file_path, engine='xlsxwriter') 
ref.to_excel(writer1, index=False)

file_path=os.path.join(output_folder, 'NewRules_scenario_20210401.xlsx')
writer2 = pd.ExcelWriter(file_path, engine='xlsxwriter') 
new.to_excel(writer2, index=False)

writer1.save()
writer2.save()


# Reading files - Shpilje HPP

In [17]:
folder_path = os.path.join('..','Data', 'hydrological_data')
#hourly_data = os.path.join(folder_path, 'New rules Fierza 20210215.xlsx') #This dataframe represnts actual data measured by operatores
#df = pd.read_excel(hourly_data,sheet_name='opr_measure')
opr_rules2 = os.path.join(folder_path, 'New rules Shpilje 20210305.xlsx') #This dataframe represnts modeling data. Becarful when you compare it with the other dataframe.
df_opr2 = pd.read_excel(opr_rules2,sheet_name='weekly')
hist_rules_path = os.path.join(folder_path, 'Shpilje_historical_rules2021416.xlsx') #This dataframe represnts modeling data. Becarful when you compare it with the other dataframe.
hist_rules = pd.read_excel(hist_rules_path)
curve_path = os.path.join(folder_path, 'level_volume_curves.xlsx')
spil_curve = pd.read_excel(curve_path, sheet_name='spilje')

In [18]:
df_opr2.head()

,month,week,spilje_outflow
0,1,1,59.358946
1,1,2,66.773720
2,1,3,49.380192
3,1,4,51.092864
4,1,5,54.694783


In [19]:
spil_curve

,Level(m),Volume(MCM)
0,490,0.00
1,495,0.60
2,500,4.90
3,505,12.40
4,510,23.10
5,520,23.15
6,530,92.40
7,540,143.90
8,550,209.10
9,560,288.90


In [20]:
#maping the monthly water level
hist_level = dict(hist_rules[['month', 'hist_level(m)']].values)
df_opr2['hist_level(m)'] = df_opr2['month'].round(0).map(hist_level)


In [21]:
# reading the storage volume based on the historical water levels

level_predict = np.poly1d(np.polyfit(spil_curve['Level(m)'], spil_curve['Volume(MCM)'], 2))
df_opr2['hist_storage(MCM)'] = level_predict(df_opr2['hist_level(m)'].round(0))


In [22]:
df_opr2['max_level(m)'] = 585
df_opr2['max_storage(MCM)'] = 520
df_opr2['hist_buffer(MCM)'] = df_opr2['max_storage(MCM)'] - df_opr2['hist_storage(MCM)']

In [23]:
df_opr2.columns

Index(['month', 'week', 'spilje_outflow', 'hist_level(m)', 'hist_storage(MCM)',
       'max_level(m)', 'max_storage(MCM)', 'hist_buffer(MCM)'],
      dtype='object')

In [25]:
levels = [5, 10, 15, 20]
Fierza_power = 500
spilje_power = 84
density = 1000 #(kg/m3) = 
g = 9.81 #(m/sec2) 
Turb_eff = 0.9 #
fierza_lower_level = 176
spilje_lower_level = 485.75 # this is the turbine elevation

for level in levels:  
    # Create the new dataframe
    _df = df_opr2[['month', 'week', 'max_level(m)','hist_level(m)', 'max_storage(MCM)', 'hist_storage(MCM)','hist_buffer(MCM)']].copy()
    
    # Calculating new buffer and storage
    _df['new_buffer(MCM)'] = _df['hist_buffer(MCM)'] + (_df['hist_buffer(MCM)']*(level/100))
    _df.loc[(_df['month']>4) & (_df['month']<10), 'new_buffer(MCM)'] = _df['hist_buffer(MCM)']
    
    _df['new_storage(MCM)'] = _df['max_storage(MCM)'] - _df['new_buffer(MCM)']
    
    _df['changeinbuffer(MCM)'] = (_df['new_buffer(MCM)'] - _df['hist_buffer(MCM)'])/_df['hist_buffer(MCM)']
    _df['additional_buffer(MCM)'] = (_df['new_buffer(MCM)'] - _df['hist_buffer(MCM)'])
    
    # Reading the new water level based on the volume-elevation curve
    level_predict = np.poly1d(np.polyfit(spil_curve['Volume(MCM)'], spil_curve['Level(m)'], 3))
    _df['new_level(m)'] = level_predict(_df['new_storage(MCM)'].round(0))
    _df.loc[(_df['month']>4) & (_df['month']<10), 'new_level(m)'] = _df['hist_level(m)']
    
    _df['lowerlevel(m)'] = spilje_lower_level
    
    # Add the calculations of X for each week
    _df['hist_head(m)'] = _df['hist_level(m)'] - _df['lowerlevel(m)']
    _df['hist_Q(m/sec)'] = ((spilje_power*1000000)/(density*g*Turb_eff*_df['hist_head(m)']))
    _df['hist_flow(MCM)'] = (_df['hist_Q(m/sec)']*3600*8760)/1000000
    _df['hist_energy(PJ)'] = ((spilje_power/1000)*31.536)
    _df['hist_X(MCM/PJ)'] = _df['hist_flow(MCM)']/_df['hist_energy(PJ)']
    
    _df['new_head(m)'] = _df['new_level(m)'] - _df['lowerlevel(m)']
    _df['new_Q(m/sec)'] = ((spilje_power*1000000)/(density*g*Turb_eff*_df['new_head(m)']))
    _df['new_flow(MCM)'] = (_df['new_Q(m/sec)']*3600*8760)/1000000
    _df['new_energy(PJ)'] = ((spilje_power/1000)*31.536)
    _df['new_X(MCM/PJ)'] = _df['new_flow(MCM)']/_df['new_energy(PJ)']
    _df['mode'] = 0
    
    # Generate the outputfile that goes to osemosys
    hist_cap = _df[['month','week','hist_storage(MCM)']]
    new_cap = _df[['month','week','new_storage(MCM)']]
    hist_x = _df[['week','mode','hist_X(MCM/PJ)']]
    new_x = _df[['week','mode','new_X(MCM/PJ)']]
    rule = _df.groupby('month')[['hist_level(m)','new_level(m)','additional_buffer(MCM)']].mean().round(1).reset_index()
    rule['diff'] = rule['hist_level(m)'] - rule['new_level(m)']
    
    #res_cap = ref[['week','Utilized Volume (MCM)']]
    for i in range(20,56):
        hist_cap['20{}'.format(i)]=hist_cap['hist_storage(MCM)']
        new_cap['20{}'.format(i)]=new_cap['new_storage(MCM)']
    hist_cap.drop('hist_storage(MCM)', axis=1, inplace=True)
    hist_cap.rename({'2055':'2055:=','week':' '},axis=1, inplace=True)
    new_cap.drop('new_storage(MCM)', axis=1, inplace=True)
    new_cap.rename({'2055':'2055:=','week':' '},axis=1, inplace=True)
    
    
    # Save files
    output_folder = os.path.join('..','processed_data', 'flood protection scenario', f'{level}%_change_buffer')
    os.makedirs(output_folder, exist_ok = True)

    file_path2=os.path.join(output_folder, f'Spilje_at_{level}%buffer.xlsx')
    writer = pd.ExcelWriter(file_path2, engine='xlsxwriter')
    
    
    _df.round(2).to_excel(writer, index=False, sheet_name='all')
    hist_cap.round(2).to_excel(writer, index=False, sheet_name='hist_cap')
    new_cap.round(2).to_excel(writer, index=False, sheet_name='new_cap')
    hist_x.round(2).to_excel(writer, index=False, sheet_name='hist_x')
    new_x.round(2).to_excel(writer, index=False, sheet_name='new_x')
    rule.round(2).to_excel(writer, index=False, sheet_name='rule')

    writer.save()
    

In [25]:
    output_folder = os.path.join('..','processed_data', 'flood protection scenario', f'{level}%_change_buffer')
    os.makedirs(output_folder, exist_ok = True)

    file_path=os.path.join(output_folder, f'Fierza_at_{level}%buffer.xlsx')
    writer = pd.ExcelWriter(file_path, engine='xlsxwriter')
    
    
    _df.round(2).to_excel(writer, index=False, sheet_name='all')
    hist_cap.round(2).to_excel(writer, index=False, sheet_name='hist_cap')
    new_cap.round(2).to_excel(writer, index=False, sheet_name='new_cap')
    hist_x.round(2).to_excel(writer, index=False, sheet_name='hist_x')
    new_x.round(2).to_excel(writer, index=False, sheet_name='new_x')
    rule.round(2).to_excel(writer, index=False, sheet_name='rule')

    writer.save()

,month,week,max_level(m),hist_level(m),max_storage(MCM),hist_storage(MCM),hist_buffer(MCM),new_buffer(MCM),new_storage(MCM),changeinbuffer(MCM),...,hist_Q(m/sec),hist_flow(MCM),hist_energy(PJ),hist_X(MCM/PJ),new_head(m),new_Q(m/sec),new_flow(MCM),new_energy(PJ),new_X(MCM/PJ),mode
0,1,1,585,569,520,379.541334,140.458666,168.550399,351.449601,0.2,...,114.283499,3604.044411,2.649024,1360.517840,78.987720,120.450384,3798.523307,2.649024,1433.933142,0
1,1,2,585,569,520,379.541334,140.458666,168.550399,351.449601,0.2,...,114.283499,3604.044411,2.649024,1360.517840,78.987720,120.450384,3798.523307,2.649024,1433.933142,0
2,1,3,585,569,520,379.541334,140.458666,168.550399,351.449601,0.2,...,114.283499,3604.044411,2.649024,1360.517840,78.987720,120.450384,3798.523307,2.649024,1433.933142,0
3,1,4,585,569,520,379.541334,140.458666,168.550399,351.449601,0.2,...,114.283499,3604.044411,2.649024,1360.517840,78.987720,120.450384,3798.523307,2.649024,1433.933142,0
4,1,5,585,569,520,379.541334,140.458666,168.550399,351.449601,0.2,...,114.283499,3604.044411,2.649024,1360.517840,78.987720,120.450384,3798.523307,2.649024,1433.933142,0
5,2,6,585,566,520,349.196297,170.803703,204.964443,315.035557,0.2,...,118.555779,3738.775044,2.649024,1411.378320,76.415970,124.504095,3926.361153,2.649024,1482.191612,0
6,2,7,585,566,520,349.196297,170.803703,204.964443,315.035557,0.2,...,118.555779,3738.775044,2.649024,1411.378320,76.415970,124.504095,3926.361153,2.649024,1482.191612,0
7,2,8,585,566,520,349.196297,170.803703,204.964443,315.035557,0.2,...,118.555779,3738.775044,2.649024,1411.378320,76.415970,124.504095,3926.361153,2.649024,1482.191612,0
8,2,9,585,566,520,349.196297,170.803703,204.964443,315.035557,0.2,...,118.555779,3738.775044,2.649024,1411.378320,76.415970,124.504095,3926.361153,2.649024,1482.191612,0
9,3,10,585,567,520,359.170331,160.829669,192.995602,327.004398,0.2,...,117.096631,3692.759351,2.649024,1394.007510,77.279091,123.113524,3882.508094,2.649024,1465.637191,0


In [50]:
# These calculations are already included in the previous step

shpi_power = 84
density = 1000 #(kg/m3) = 
g = 9.81 #(m/sec2)  
Turb_eff = 0.9 # 

In [51]:
ref = df_opr2.copy()
new = df_opr2.copy()

In [52]:
ref['Head (m)'] = ref['Net Head (m)'] 
ref['Q [m/sec]'] = ((shpi_power*1000000)/(density*g*Turb_eff*ref['Head (m)']))
ref['flow_MCM'] = (ref['Q [m/sec]']*3600*8760)/1000000
ref['energy_PJ'] = ((shpi_power/1000)*31.536)
ref['ratio_X [MCM/PJ]'] = ref['flow_MCM']/ref['energy_PJ']
#dff['res_cap [MCM]'] = ((dff['inflow [m3/s].1'].mean()*3600*8760)/1000000)

In [53]:
new['Head (m)'] = new['New Net Head (m)'] 
new['Q [m/sec]'] = ((shpi_power*1000000)/(density*g*Turb_eff*new['Head (m)']))
new['flow_MCM'] = (new['Q [m/sec]']*3600*8760)/1000000
new['energy_PJ'] = ((shpi_power/1000)*31.536)
new['ratio_X [MCM/PJ]'] = new['flow_MCM']/new['energy_PJ']


In [54]:
tts = ref[['week','ratio_X [MCM/PJ]']]
tts['m1'] = 0
tts =tts_input[['week','m1','ratio_X [MCM/PJ]']]
tts.head()

,week,m1,ratio_X [MCM/PJ]
0,1,0,1132.65
1,2,0,1132.65
2,3,0,1132.65
3,4,0,1132.65
4,5,0,1132.65


In [57]:
ref.round(2).head()

,month,week,Debar/Shpilje HPP,Net Head (m),New Net Head (m),Head (m),Q [m/sec],flow_MCM,energy_PJ,ratio_X [MCM/PJ]
0,1,1,59.36,91.3,77.6,91.3,104.21,3286.27,2.65,1240.56
1,1,2,66.77,91.3,77.6,91.3,104.21,3286.27,2.65,1240.56
2,1,3,49.38,91.3,77.6,91.3,104.21,3286.27,2.65,1240.56
3,1,4,51.09,91.3,77.6,91.3,104.21,3286.27,2.65,1240.56
4,1,5,54.69,91.3,77.6,91.3,104.21,3286.27,2.65,1240.56


In [23]:
output_folder = os.path.join('..','processed_data', 'flood protection scenario')
os.makedirs(output_folder, exist_ok = True)

file_path=os.path.join(output_folder, 'Shpilje_Reference_scenario_20210305.xlsx')
writer1 = pd.ExcelWriter(file_path, engine='xlsxwriter') 
ref.round(2).to_excel(writer1, index=False)

file_path=os.path.join(output_folder, 'Shpilje_NewRules_scenario_20210305.xlsx')
writer2 = pd.ExcelWriter(file_path, engine='xlsxwriter') 
new.round(2).to_excel(writer2, index=False)

writer1.save()
writer2.save()

# Skavica HPP

In [70]:
folder_path = os.path.join('..','Data', 'hydrological_data')
sk_file = os.path.join(folder_path, 'SkavicaOperationalRules20210308.xlsx')
sk_df = pd.read_excel(sk_file,sheet_name='inputdata')

In [71]:
sk_df.head()

,month,week,Max Level (m),Operational level (m),lower level (m),Net Head (m),Utilized Volume (MCM),Max Volume (MCM),Buffer volume (MCM)
0,1,1,420,400,300,100,700,2500,1800
1,1,2,420,400,300,100,700,2500,1800
2,1,3,420,400,300,100,700,2500,1800
3,1,4,420,400,300,100,700,2500,1800
4,1,5,420,400,300,100,700,2500,1800


In [72]:
Skavic_power = 196
Fierza_power = 500
Komani_power = 600
Vaudej_power = 250
density = 1000 #(kg/m3) = 
g = 9.81 #(m/sec2)  
Turb_eff = 0.9 # 

In [73]:
ref = sk_df.copy()


In [74]:
power = Skavic_power
ref['Head (m)'] = ref['Net Head (m)'] 
ref['Q [m/sec]'] = ((power*1000000)/(density*g*Turb_eff*ref['Head (m)'])).round(2)
ref['flow_MCM'] = ((ref['Q [m/sec]']*3600*8760)/1000000).round(2)
ref['energy_PJ'] = ((power/1000)*31.536)
ref['ratio_X [MCM/PJ]'] = (ref['flow_MCM']/ref['energy_PJ']).round(2)
#dff['res_cap [MCM]'] = ((dff['inflow [m3/s].1'].mean()*3600*8760)/1000000)

In [75]:
ref

,month,week,Max Level (m),Operational level (m),lower level (m),Net Head (m),Utilized Volume (MCM),Max Volume (MCM),Buffer volume (MCM),Head (m),Q [m/sec],flow_MCM,energy_PJ,ratio_X [MCM/PJ]
0,1,1,420,400,300,100,700,2500,1800,100,222.00,7000.99,6.181056,1132.65
1,1,2,420,400,300,100,700,2500,1800,100,222.00,7000.99,6.181056,1132.65
2,1,3,420,400,300,100,700,2500,1800,100,222.00,7000.99,6.181056,1132.65
3,1,4,420,400,300,100,700,2500,1800,100,222.00,7000.99,6.181056,1132.65
4,1,5,420,400,300,100,700,2500,1800,100,222.00,7000.99,6.181056,1132.65
5,2,6,420,400,300,100,700,2500,1800,100,222.00,7000.99,6.181056,1132.65
6,2,7,420,400,300,100,700,2500,1800,100,222.00,7000.99,6.181056,1132.65
7,2,8,420,400,300,100,700,2500,1800,100,222.00,7000.99,6.181056,1132.65
8,2,9,420,400,300,100,700,2500,1800,100,222.00,7000.99,6.181056,1132.65
9,3,10,420,415,300,115,1000,2500,1500,115,193.04,6087.71,6.181056,984.90


In [76]:
tts_input = ref[['week','ratio_X [MCM/PJ]']]
tts_input['m1'] = 0
tts_input =tts_input[['week','m1','ratio_X [MCM/PJ]']]
tts_input.head()

,week,m1,ratio_X [MCM/PJ]
0,1,0,1132.65
1,2,0,1132.65
2,3,0,1132.65
3,4,0,1132.65
4,5,0,1132.65


In [88]:
res_cap = ref[['week','Utilized Volume (MCM)']]
for i in range(20,56):
    res_cap['20{}'.format(i)]=res_cap['Utilized Volume (MCM)']

res_cap.drop('Utilized Volume (MCM)', axis=1, inplace=True)
res_cap.rename({'2055':'2055:=','week':' '},axis=1, inplace=True)
res_cap.head()

,,2020,2021,2022,2023,2024,2025,2026,2027,2028,...,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055:=
0,1,700,700,700,700,700,700,700,700,700,...,700,700,700,700,700,700,700,700,700,700
1,2,700,700,700,700,700,700,700,700,700,...,700,700,700,700,700,700,700,700,700,700
2,3,700,700,700,700,700,700,700,700,700,...,700,700,700,700,700,700,700,700,700,700
3,4,700,700,700,700,700,700,700,700,700,...,700,700,700,700,700,700,700,700,700,700
4,5,700,700,700,700,700,700,700,700,700,...,700,700,700,700,700,700,700,700,700,700


In [69]:
ref.columns

Index(['month', 'week', 'Max Level (m)', 'Operational level (m)',
       'lower level (m)', 'Net Head (m)', 'Utilized Volume (MCM) ',
       'Max Volume (MCM)', 'Buffer volume (MCM)', 'Head (m)', 'Q [m/sec]',
       'flow_MCM', 'energy_PJ', 'ratio_X [MCM/PJ]'],
      dtype='object')

In [89]:
output_folder = os.path.join('..','processed_data', 'skavica scenario')
os.makedirs(output_folder, exist_ok = True)

file_path=os.path.join(output_folder, 'Skavica_Reference_scenario_20210414.xlsx')
writer1 = pd.ExcelWriter(file_path, engine='xlsxwriter') 
ref.round(2).to_excel(writer1, index=False, sheet_name='all')
tts_input.round(0).to_excel(writer1, index=False, sheet_name='tts_input')
res_cap.round(0).to_excel(writer1, index=False, sheet_name='residual_capacity')

writer1.save()
